In [1]:
#Importing the required packages I am using in this project
import os
import pandas as pd
import numpy as np

In [4]:
#Setting the working directory to import the file
os.chdir("/Users/praneeth/Downloads/OfficeWork")

In [5]:
#Checking the final working directory
print(os.getcwd())

/Users/praneeth/Downloads/OfficeWork


In [6]:
#Importing the python file
df = pd.read_excel("CreditUnion.xlsx")

In [7]:
#Printing the first 5 rows to check if the import process is successful
df.head(5)

,FID,CU_NUMBER,NAME,CITY,Total Assets,SITE_ID,ADDRESS,STATE,ZIP,TELEPHONE,TYPE,STATUS,COUNTY,COUNTYFIPS,LONGITUDE
0,5001,6527,BRIGHTVIEW,Ridgeland,NaN,37338,713 S Pear Orchard Rd Ste 101,MS,39157,6019779470,Corporate Office,Active,Madison,28089,-90.123535
1,5002,6527,BRIGHTVIEW,Tallahassee,NaN,44140,1585 Summit Lake Dr,FL,32317,8509429190,Other,Active,Leon,12073,-84.149281
2,5003,6528,S.A.I.F.,Baton Rouge,NaN,30493,10190 Celtic Dr,LA,70809,2252925490,Corporate Office,Active,East Baton Rouge,22033,-91.075009
3,5004,6528,S.A.I.F.,Birmingham,NaN,30520,7400 Cahaba Valley Rd,AL,35242,8002994410,Other,Active,Shelby,1117,-86.661479
4,5005,6529,CAPITAL CITY PRESS,Port Allen,NaN,45131,1274 Commercial Dr,LA,70767,2253837430,Other,Active,West Baton Rouge,22121,-91.214907


In [13]:
#I will check for the nul values
df.isnull().sum()

order_id                 0
Borough                  0
customer_id              0
restaurant_name          0
cuisine_type             0
cost_of_the_order        0
day_of_the_week          0
rating                   0
food_preparation_time    0
delivery_time            0
dtype: int64

In [14]:
#We do not require the values that have "Not given". We will check the number
count = (df['rating'] == 'Not given').sum()
print(count)

736


In [15]:
#We cannot get any inference from them. Hence, we remove them. 
df = df[df['rating'] != 'Not given']

In [16]:
count = (df['rating'] == 'Not given').sum()
print(count)

0


In [17]:
df.describe()

,order_id,customer_id,cost_of_the_order,food_preparation_time,delivery_time
count,1.162000e+03,1162.000000,1162.000000,1162.000000,1162.000000
mean,1.477520e+06,171939.471601,16.760766,27.381239,24.154045
std,5.468713e+02,114551.215031,7.572578,4.677922,4.930999
min,1.476547e+06,5139.000000,4.470000,20.000000,15.000000
25%,1.477062e+06,77540.000000,12.130000,23.000000,20.000000
50%,1.477524e+06,128655.500000,14.600000,27.000000,25.000000
75%,1.477992e+06,276066.250000,22.750000,32.000000,28.000000
max,1.478444e+06,403019.000000,35.410000,35.000000,33.000000


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1162 entries, 2 to 1896
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   order_id               1162 non-null   int64  
 1   Borough                1162 non-null   object 
 2   customer_id            1162 non-null   int64  
 3   restaurant_name        1162 non-null   object 
 4   cuisine_type           1162 non-null   object 
 5   cost_of_the_order      1162 non-null   float64
 6   day_of_the_week        1162 non-null   object 
 7   rating                 1162 non-null   object 
 8   food_preparation_time  1162 non-null   int64  
 9   delivery_time          1162 non-null   int64  
dtypes: float64(1), int64(4), object(5)
memory usage: 99.9+ KB


In [23]:
import pandas as pd
average_df = df.groupby('restaurant_name').agg({
    'cost_of_the_order': 'mean',
    'rating': 'mean',
    'food_preparation_time': 'mean',
    'delivery_time': 'mean'
}).reset_index()

# Merge the calculated averages with the original DataFrame
merged_df = df.drop(columns=['cost_of_the_order', 'rating', 'food_preparation_time', 'delivery_time','day_of_the_week','customer_id','order_id'])
merged_df = merged_df.merge(average_df, on='restaurant_name', how='left')

# Rename the columns to match the original names
merged_df = merged_df.rename(columns={
    'cost_of_the_order': 'average_cost_of_the_order',
    'rating': 'average_rating',
    'food_preparation_time': 'average_food_preparation_time',
    'delivery_time': 'average_delivery_time'
})

new_df = merged_df.drop_duplicates()
# Now, 'merged_df' contains the requested averages with the original columns


In [25]:
new_df

,Borough,restaurant_name,cuisine_type,average_cost_of_the_order,average_rating,average_food_preparation_time,average_delivery_time
0,Brooklyn,Cafe Habana,Mexican,12.400909,4.272727,26.090909,26.818182
1,Staten Island,Blue Ribbon Fried Chicken,American,17.665469,4.328125,26.984375,24.171875
2,Staten Island,Dirty Bird to Go,American,15.495000,4.000000,25.000000,21.500000
3,Bronx,Tamarind TriBeCa,Indian,16.140500,4.400000,26.800000,22.400000
4,Manhattan,Barbounia,Mediterranean,14.262500,4.500000,30.250000,23.000000
...,...,...,...,...,...,...,...
1137,Queens,DuMont Burger,American,26.650000,4.000000,25.000000,25.000000
1139,Queens,Sushi of Gari 46,Japanese,18.005882,4.235294,28.705882,24.529412
1149,Bronx,'wichcraft,American,8.100000,5.000000,28.000000,28.000000
1152,Bronx,Melt Shop,American,17.693333,4.333333,27.333333,22.333333


Scenario 1:  I work in New York City in Manhattan. I need a place where I can have food on a regular basis. For this I want a place where I can have food for less than or equal to 15$ per meal. The place should be present in Manhattan if every I had to pick the food myself. The delivery time should be under 25 mins.

In [29]:
#Solving Scenario 1

criteria1 = (new_df['average_cost_of_the_order'] <= 15) & (new_df['Borough'] == 'Manhattan')

df1 = new_df[criteria1]

df1 = df1.sort_values(by="average_delivery_time", ascending=True)

df1.head(5)


,Borough,restaurant_name,cuisine_type,average_cost_of_the_order,average_rating,average_food_preparation_time,average_delivery_time
299,Manhattan,Wo Hop Restaurant,Chinese,14.160,4.0,22.0,16.0
861,Manhattan,Big Daddy's,American,13.460,5.0,27.0,19.0
810,Manhattan,Lantern Thai Kitchen,Thai,9.460,5.0,29.0,19.0
800,Manhattan,Hampton Chutney Co.,Thai,12.370,4.0,21.0,20.0
293,Manhattan,Terakawa Ramen,Japanese,11.595,3.5,28.5,20.5


In [31]:
#Solving Scenario 2

criteria2 = (new_df['cuisine_type'] == "Japanese") & (new_df["average_rating"] == 5)

df2 = new_df[criteria2]

df2 = df2.sort_values(by="average_food_preparation_time", ascending=True)

df2.head(5)

,Borough,restaurant_name,cuisine_type,average_cost_of_the_order,average_rating,average_food_preparation_time,average_delivery_time
22,Queens,Haru Gramercy Park,Japanese,29.8300,5.0,20.00,32.0
156,Brooklyn,Samurai Mama,Japanese,11.6400,5.0,21.00,18.0
614,Brooklyn,Asuka Sushi,Japanese,21.1000,5.0,24.00,27.5
1011,Manhattan,Asuka Sushi,Japanese,21.1000,5.0,24.00,27.5
112,Manhattan,Yama 49,Japanese,19.8125,5.0,24.25,26.0
